<a href="https://colab.research.google.com/github/A-Burnhard/Credit_Scoring_ML_Data/blob/main/Credit_Rating_Data_Model_Nerasol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive//Machine Learning/Credit Data/train.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read Dataset**

In [ ]:
df = pd.read_csv(file_path)
df.head()

<ipython-input-161-e35bec9b854a>:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


**Explore Dataset**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [ ]:
df

In [ ]:
df = df.drop(['ID','Age','Interest_Rate','Payment_of_Min_Amount','Occupation','Num_Bank_Accounts','Num_Credit_Card','Credit_Utilization_Ratio','Num_Credit_Inquiries','Customer_ID','Month','Name','SSN',
               'Type_of_Loan','Total_EMI_per_month', 'Annual_Income', 'Changed_Credit_Limit','Credit_Mix','Delay_from_due_date','Monthly_Balance', 'Credit_History_Age','Amount_invested_monthly','Outstanding_Debt' ], axis = 1)

In [ ]:
df.describe(include='object').T

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

(100000, 6)

In [ ]:
df

# Data Cleaning


**Num of delayed payment**

In [ ]:
df['Num_of_Delayed_Payment'].isna().count()

np.int64(100000)

In [ ]:
df['Num_of_Delayed_Payment'] = pd.to_numeric(df['Num_of_Delayed_Payment'], errors='coerce')


In [ ]:
df[df['Num_of_Delayed_Payment'] < 0]['Num_of_Delayed_Payment'].count()


np.int64(622)

In [ ]:
df.drop(df[df['Num_of_Delayed_Payment'] < 0].index,inplace=True)


In [ ]:
df['Num_of_Delayed_Payment'].isna().sum()

np.int64(9746)

In [ ]:
from sklearn.impute import KNNImputer
knn=KNNImputer(n_neighbors=3,weights='distance')

In [ ]:
df[['Num_of_Delayed_Payment']] = knn.fit_transform(df[['Num_of_Delayed_Payment']])

In [ ]:
df['Num_of_Delayed_Payment']=df['Num_of_Delayed_Payment'].astype(int)

**Number of Loans**

In [ ]:
df['Num_of_Loan'].value_counts(ascending=False).head(10)

In [ ]:
df['Num_of_Loan'] = pd.to_numeric(df['Num_of_Loan'], errors='coerce')

In [ ]:
nl=df[df['Num_of_Loan'] > 60]['Num_of_Loan'].unique()
nl.size

367

In [ ]:
df[df['Num_of_Loan'] < 0]['Num_of_Loan'].unique()


array([-100.])

In [ ]:
df.drop(df[df['Num_of_Loan'] > 60 ].index,inplace=True)
df.drop(df[df['Num_of_Loan'] < 0 ].index,inplace=True)

In [ ]:
from sklearn.impute import KNNImputer
knn=KNNImputer(n_neighbors=3,weights='distance')

In [ ]:
df[['Num_of_Loan']]=knn.fit_transform(df[['Num_of_Loan']])


In [ ]:
df['Num_of_Loan']=df['Num_of_Loan'].astype(int)


**Monthly In hand Salary**

**Credit Score**

In [ ]:
df['Credit_Score'].value_counts()

In [ ]:
target_mapping = {'Poor': 0, 'Standard': 1, 'Good': 2}
df['Credit_Score'] = df['Credit_Score'].map(target_mapping)
df['Credit_Score'].value_counts()

In [ ]:
df

,Monthly_Inhand_Salary,Num_of_Loan,Num_of_Delayed_Payment,Payment_Behaviour,Credit_Score
0,1824.843333,4,7,High_spent_Small_value_payments,2
1,NaN,4,31,Low_spent_Large_value_payments,2
2,NaN,4,7,Low_spent_Medium_value_payments,2
3,NaN,4,4,Low_spent_Small_value_payments,2
4,1824.843333,4,31,High_spent_Medium_value_payments,2
...,...,...,...,...,...
99995,3359.415833,2,7,High_spent_Large_value_payments,0
99996,3359.415833,2,7,High_spent_Medium_value_payments,0
99997,3359.415833,2,6,High_spent_Large_value_payments,0
99998,3359.415833,2,31,Low_spent_Large_value_payments,1


****

**Save Data**

In [ ]:
df.to_csv("train_cleaned.csv",index=False)